#### 1. 本报告定义了如何快速实现部分alpha因子值
#### 2. 对于报告中的难点进行了讨论和探究
---

#### 1. paper中每个alpha计算公式用到了基本的开高低收价格及成交量、成交额等行情数据
#### 2. 通过对投资组合中的每只股票按其公式计算对应的alpha因子值，然后进行排序评分，作为下一期的投资参考
#### 3. 具体的投资策略，可根据计算出的因子值灵活构建，如可根据因子值进行每只股票的持仓权重设定等
---

#### 公式实现框架概述

* 以 600029.SH, 601398.SH, 600030.SH ,600010.SH ,'600031.SH' 5只股票为例，定义一个简易的投资组合，探讨如何快速的实现因子值

* abs(x) ,log(x) ,ts_min(x,d), ts_max(x,d), ts_argmax(x,d), ts_argmin(x,d),correlation(x,y,d),delay(x,d) 等等基础计算公式可通过 **pandas ,numpy** 自带的函数实现  
* 实现难点在于 **rank** 函数，论文中的rank(x) 表示某股票x值在股票池中横截面上的升序排名序号，并将排名归一到[0,1]的闭区间  
* 对于含有 **rank**函数的公式可具体细分如下：
> 1. 若只有一层rank:
>> 若rank(x) 中的x 变量为volume、open、close这类行情数据中已有的项目，则可通过传入已有项的名称来实现  
>> 若rank(x) 中x 变量是某个量价因子所产生的值，可通过用户自定义x的计算公式，传入函数参数实现   (未实现)
> 2. 若含有多层rank:  
>> 如rank(function(rank(x)))等形式，只能用户自定义函数逐步实现 (未实现)

In [1]:
import pandas as pd
import numpy as np
from wlkbacktest import Strategy

In [2]:
# 配置策略环境
def init(context):
    context.start = '20080101'
    context.end = '20151001'
    context.securities = ['600029.SH','601398.SH','600030.SH','600010.SH','600031.SH'] # 以5只股票为例，设定一个投资组合
    
stra = Strategy(init) #初始化策略类
datas = stra.get_trading_data() #获取交易数据

In [3]:
datas # 所有交易数据

<class 'pandas.core.panel.Panel'>
Dimensions: 5 (items) x 1885 (major_axis) x 6 (minor_axis)
Items axis: 600010.SH to 601398.SH
Major_axis axis: 2008-01-02 00:00:00 to 2015-09-30 00:00:00
Minor_axis axis: open to amount

### 查阅一下数据

In [4]:
datas['600029.SH'].head()

,open,high,close,low,volume,amount
date,,,,,,
2008-01-02,17.1400,17.9900,17.5500,17.0300,26385079.0000,759282152.0000
2008-01-03,17.3600,17.3700,17.0100,16.6100,23295654.0000,644080406.0000
2008-01-04,17.0700,17.1900,16.8000,16.3700,29081275.0000,795181029.0000
2008-01-07,16.8300,16.9600,16.8100,16.3600,20952882.0000,569655271.0000
2008-01-08,16.8600,17.1000,16.2700,16.0600,27145581.0000,732970680.0000


In [5]:
datas['600010.SH'].tail()

,open,high,close,low,volume,amount
date,,,,,,
2015-09-24,3.7000,3.7800,3.7500,3.7000,55137572.0000,206567904.0000
2015-09-25,3.7300,3.7800,3.6400,3.6100,60486856.0000,223005776.0000
2015-09-28,3.6700,3.7000,3.6600,3.5900,34519800.0000,125944416.0000
2015-09-29,3.5900,3.6000,3.5300,3.5000,66740496.0000,236192896.0000
2015-09-30,3.5500,3.5900,3.5500,3.5100,53250280.0000,188991264.0000



#### 举例说明
* 以下分别就含rank和不含rank的公式进行举例说明  

#### 以Alpha 6 和 Alpha 12 两个不含rank的公式为例:
* Alpha 6：   -1 * correlation(open, volume, 10)
* Alpha 12：  sign(delta(volume, 1)) ** (-1 * delta(close, 1))

#### 以Alpha 44 含一层rank(x)的公式为例 且 x 为行情数据已有项:
* Alpha 44：   -1 * correlation(high, rank(volume), 5)

In [6]:
# Alpha 6
def alpha_6(data, **kwargs):
    '''
    param data: pandas.DataFrame;including open,high,low,close,volume,amount
    '''
    data = data.copy()
    res = -pd.rolling_corr(data['open'], data['volume'], window=10)
    return res

# Alpha 12
def alpha_12(data, **kwargs):
    data = data.copy()
    res = np.sign(data['volume'].diff()) * (-1 * data['close'].diff())
    return res

# Alpha 44
def alpha_44(data):
    '''
    data: pandas.DataFrame 
    data中的r_volume值交由外层的函数实现
    假设data参数已默认包含'r_volume'列,方便公式逻辑的实现
    '''
    data = data.copy()
    res = - pd.rolling_corr(data['high'], data['r_volume'], 5)
    return res

### add_alpha快速实现因子值的计算，传入行情数据参数，和需计算的alpha公式函数，及函数中所需要涉及到rank的项

** datas 为原始的交易数据，pandas.Panel结构，每个标的名称对应的DataFrame只含开高低收，成交量，成交额6项数据 **    
** funs 函数列表传入 需要计算的alpha公式函数名 **  
** rank_items 为公式中需要涉及rank（x）的项 **

In [7]:
# 定义添加alpha函数
def add_alpha(datas,funs,rank_items = []):
    '''
    params : list of functions
    params : 要参与排名的itesm e.g.'open','volume'，以实现funs里面函数所需的 'r_volume','r_open'等项
    '''
    secs = datas.items # 标的个数
    nums = len(secs)
    if len(rank_items)!=0: #若要排序的项不为0
        datas_ = datas.swapaxes('items','minor') #转换格式，方便比较
        for item in rank_items:
            x = (datas_[item].rank(1) - 1)/nums
            for sec in secs:
                datas[sec]['r_'+item] = x[sec]

    for sec in secs:
        df = datas[sec]
        for fun in funs:
            df[fun.__name__] = fun(df)
    return datas

In [8]:
new_datas = add_alpha(datas,funs=[alpha_6,alpha_12,alpha_44],rank_items=['open','volume'])

In [9]:
new_datas

<class 'pandas.core.panel.Panel'>
Dimensions: 5 (items) x 1885 (major_axis) x 6 (minor_axis)
Items axis: 600010.SH to 601398.SH
Major_axis axis: 2008-01-02 00:00:00 to 2015-09-30 00:00:00
Minor_axis axis: open to amount

## 查看数据 以600030.SH为例

In [10]:
new_datas['600030.SH'].tail(20)

,open,high,close,low,volume,amount,r_open,r_volume,alpha_6,alpha_12,alpha_44
date,,,,,,,,,,,
2015-09-01,14.6000,15.3100,15.1900,14.2000,498759904.0000,7443513344.0000,0.8000,0.4000,0.9144,0.1200,-0.4939
2015-09-02,14.6200,15.1500,14.8700,14.5100,362296000.0000,5389001216.0000,0.8000,0.4000,0.8475,-0.3200,-0.6294
2015-09-07,14.8500,15.4400,14.7600,14.7000,200888128.0000,3032953856.0000,0.8000,0.4000,0.4947,-0.1100,-0.5676
2015-09-08,14.6000,15.2900,15.2400,14.2800,215004144.0000,3183032832.0000,0.8000,0.6000,0.0489,-0.4800,0.0489
2015-09-09,15.3400,15.7900,15.6800,15.1900,328061664.0000,5101934080.0000,0.8000,0.6000,-0.4091,-0.4400,-0.5408
2015-09-10,15.4000,15.5100,15.1500,15.0700,192552624.0000,2949927680.0000,0.8000,0.8000,-0.0685,-0.5300,-0.4399
2015-09-11,15.1700,15.4000,15.0200,14.9200,141370912.0000,2142975360.0000,0.8000,0.8000,-0.0104,-0.1300,0.0255
2015-09-14,15.0800,15.1300,13.9100,13.5800,262223664.0000,3739513600.0000,0.8000,0.4000,-0.0023,1.1100,-0.4287
2015-09-15,13.7500,14.1900,13.5200,13.4300,170885888.0000,2354542848.0000,0.8000,0.4000,-0.1225,-0.3900,-0.6472


## 查看数据 601398.SH为例

In [11]:
new_datas['601398.SH'].tail(20)

,open,high,close,low,volume,amount,r_open,r_volume,alpha_6,alpha_12,alpha_44
date,,,,,,,,,,,
2015-09-01,3.9800,4.3000,4.2700,3.9600,1018541248.0000,4491813888.0000,0.2000,0.8000,0.4582,-0.2900,-0.2272
2015-09-02,4.1500,4.7000,4.7000,4.1200,1394605696.0000,6542697984.0000,0.2000,0.8000,0.1058,-0.4300,-0.5725
2015-09-07,4.4900,4.5100,4.2900,4.2700,648542784.0000,2988526848.0000,0.2000,0.8000,-0.0894,-0.4100,-0.8326
2015-09-08,4.2200,4.3100,4.2700,4.1300,404199392.0000,1800877184.0000,0.2000,0.8000,-0.1202,-0.0200,-0.7779
2015-09-09,4.2300,4.3000,4.2700,4.2000,366216704.0000,1636333824.0000,0.2000,0.8000,-0.0804,0.0000,0.0000
2015-09-10,4.2400,4.2700,4.2500,4.2100,184433632.0000,822389632.0000,0.2000,0.6000,0.0496,-0.0200,-0.4496
2015-09-11,4.2300,4.2500,4.2100,4.2000,128608432.0000,572234112.0000,0.2000,0.6000,0.1112,-0.0400,-0.5940
2015-09-14,4.2000,4.3700,4.2900,4.1500,426149920.0000,1920341376.0000,0.2000,0.8000,0.1215,-0.0800,-0.7968
2015-09-15,4.2500,4.3500,4.3100,4.2200,382841472.0000,1729309184.0000,0.2000,0.8000,0.1995,0.0200,-0.8561


** add_alpha 函数可以快速实现因子值计算 **  
** 同时方便编写只含一个rank(x)(x为 open,high,low,close,volume,amount之一) 的ahpha公式函数，只需在函数中假设data参数已含有 rank(x)这项列值即可，名称为 'r_XXXX' ,例如 'r_volume' 代表 rank(volume)的值**